# CorrelAid for Klima*Kollektiv
Aug 17, 2024

by Nicolas Fröhlich, partly based on Florian Detsch and ChatGPT

In [2]:
# !pip install geopandas
# !pip install folium
# !pip install requests

     |████████████████████████████████| 108 kB 1.5 MB/s eta 0:00:01


In [318]:
import pandas as pd
import geopandas as gpd
import folium
import requests
from shapely.geometry import Polygon, LineString

In [10]:
# ---- Scrape consumers and identify large ones ----

# Read geojson of Wasserverbraucher (only run once, please)
url = "https://www.klaerwerk-krefeld.org/wasserbuch/utm.php"
data = requests.get(url).json()

In [320]:
# Save to GeoDataFrame
consumer_all = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

# Save locally as GeoJSON
consumer_all.to_file("consumer_all.geojson", driver='GeoJSON')

# Convert 'size' to numeric
consumer_all['size'] = pd.to_numeric(consumer_all['size'], errors='coerce')

# Identify large consumers (more than 1 mio m3/a of water granted)
consumer = consumer_all[consumer_all['size'] > 1000000]


In [322]:
# ---- Spatial subset with bounding box ----

# Define coordinates of the rectangle [Köln, Mettmann, Roermond, Aachen]
coords = [(5.9512135386, 50.7489864309),
          (7.0077544451, 50.7489864309),
          (7.0077544451, 51.2670019108),
          (5.9512135386, 51.2670019108),
          (5.9512135386, 50.7489864309)]  # Close the polygon by repeating the first point

# Create a polygon from the coordinates
bounding_box = Polygon(coords)

# Create a GeoDataFrame for the bounding box
bounding_box_gdf = gpd.GeoDataFrame(index=[0], crs=consumer.crs, geometry=[bounding_box])

# Find the points that intersect with the polygon
consumer_within = consumer[consumer.intersects(bounding_box)]

# Create a LineString from coordinates
line = LineString(coords)

# Convert the LineString to a GeoJSON format
bounding_box = json.loads(json.dumps(line.__geo_interface__))

In [324]:
# ---- Visualization ----

# Create a base map
m = folium.Map(location=[51.0, 6.5], zoom_start=9)

# Add bounding box linestring
folium.GeoJson(
    bounding_box,
    style_function=lambda x: {'color': 'red', 'weight': 2},
    name='Bounding Box' # determine the name for the layer toggle
).add_to(m)

# Create a FeatureGroup for consumer points
consumers_group = folium.FeatureGroup(name='Consumer Points')

# Add consumer points to the FeatureGroup
for idx, row in consumer_within.iterrows():
    title_part = row['title'].split('/', 1)[1] if '/' in row['title'] else row['title']
    popup_text = f"{title_part} consumes {row['size']} m3 of water"
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        popup=folium.Popup(html=popup_text, max_width=200),
        color='blue',
        fill=True
    ).add_to(consumers_group)

# Add the FeatureGroup to the map
consumers_group.add_to(m)

In [340]:
# load GeoPackage file 'transportleitungen_geom' into geodataframe gdf
gdf = gpd.read_file("transportleitung_geom.gpkg")

# print to inspect the data
gdf

,id,Name,geometry
0,0,transportleitung_west,"MULTILINESTRING ((350531.177 5662445.558, 3501..."
1,1,transportleitung_sued,"MULTILINESTRING ((333215.541 5658810.732, 3341..."


In [354]:
gdf = gdf.to_crs(consumer.crs) # change CRS to the same as in 'consumer'

# check that all have same CRS
print(f'CRSystems are {bounding_box_gdf.crs}, {gdf.crs}, and {consumer.crs}.')

CRSystems are EPSG:4326, EPSG:4326, and EPSG:4326.


In [334]:

# Add the GeoDataFrame to the map
folium.GeoJson(gdf, 
               name="Water pipelines", # again, determine the name for the layer toggle
               style_function=lambda feature: {'color': 'orange' if feature['properties']['Name'] == 'transportleitung_sued' else 'green'},
               popup=folium.GeoJsonPopup(fields=['Name'])
              ).add_to(m)

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

In [336]:
m

In [338]:
# save the map as hmtl
m.save('map.html')

---

### Play around with Sentinel 2 satellite data as potential background 

Data source:
https://zipper.dataspace.copernicus.eu/odata/v1/Products(25563de0-bfd5-4661-96e6-aeeef7c81917)/$value

In [84]:
import os

# Set the directory path
directory_path = '/Users/nf/docs/uni/DataStuff/Spatial/data/S2A_MSIL1C_20240824T104021_N0511_R008_T31UGS_20240824T142025.SAFE/GRANULE/L1C_T31UGS_A047910_20240824T104047/IMG_DATA'

# Initialize an empty list to store file paths
files = []

# Walk through the directory
for filename in os.listdir(directory_path):
    # Check if the file is a regular file (not a directory)
    if os.path.isfile(os.path.join(directory_path, filename)):
        # Append the full path of the file to the list
        files.append(os.path.join(directory_path, filename))

# Print the first 3 files to verify
print(files[:3])


['/Users/nf/docs/uni/DataStuff/Spatial/data/S2A_MSIL1C_20240824T104021_N0511_R008_T31UGS_20240824T142025.SAFE/GRANULE/L1C_T31UGS_A047910_20240824T104047/IMG_DATA/T31UGS_20240824T104021_B11.jp2', '/Users/nf/docs/uni/DataStuff/Spatial/data/S2A_MSIL1C_20240824T104021_N0511_R008_T31UGS_20240824T142025.SAFE/GRANULE/L1C_T31UGS_A047910_20240824T104047/IMG_DATA/T31UGS_20240824T104021_B05.jp2', '/Users/nf/docs/uni/DataStuff/Spatial/data/S2A_MSIL1C_20240824T104021_N0511_R008_T31UGS_20240824T142025.SAFE/GRANULE/L1C_T31UGS_A047910_20240824T104047/IMG_DATA/T31UGS_20240824T104021_B04.jp2']


In [102]:
import rasterio

for file in files:
    # Open the JPEG 2000 file
    with rasterio.open(file) as src:
        array = src.read(1)  # Read the first band

        print(array.shape)  # Print the shape of the array


(5490, 5490)
(5490, 5490)
(10980, 10980)
(1830, 1830)
(5490, 5490)
(5490, 5490)
(5490, 5490)
(10980, 10980)
(10980, 10980)
(1830, 1830)
(5490, 5490)
(10980, 10980)
(1830, 1830)
(10980, 10980)
